# Generation of BoM Data for Supply Chain Simulation

In [1]:
SAVE_RECORDS=True

## Supply Chain Simulation

In [2]:
import numpy as np
from scipy import interpolate
import math
import random

class Edge:
    def __init__(self, unit_price, min_cost, max_cost):
        self.unit_price = unit_price
        self.in_transit = [0, 0, 0]  # 3 cycles of transit
        self.quantity = 0  # Current cycle's quantity
        self.min_cost = min_cost
        self.max_cost = max_cost
        self.current_cost = 0

    def update(self, new_quantity):
        delivered = self.in_transit.pop(0)
        self.in_transit.append(new_quantity)
        self.quantity = delivered
        return delivered

    def initialize(self, initial_quantity):
        self.in_transit = [initial_quantity] * 3
        self.quantity = initial_quantity

    def calculate_cost(self, source_node):
        if source_node.cost_type == "fixed":
            self.current_cost = (self.min_cost + self.max_cost) / 2
        elif source_node.cost_type == "positive_dynamic":
            cost_range = self.max_cost - self.min_cost
            inventory_ratio = source_node.inventory / source_node.max_inventory
            self.current_cost = self.min_cost + (cost_range * inventory_ratio)
        elif source_node.cost_type == "negative_dynamic":
            cost_range = self.max_cost - self.min_cost
            inventory_ratio = 1 - (source_node.inventory / source_node.max_inventory)
            self.current_cost = self.min_cost + (cost_range * inventory_ratio)
        return self.current_cost * self.quantity

class Node:
    def __init__(self, max_inventory, node_class, cost_type):
        self.max_inventory = max_inventory
        self.inventory = 0
        self.node_class = node_class
        self.incoming_edges = []
        self.outgoing_edges = []
        self.last_production = 0
        self.cost_type = cost_type

    def add_incoming_edge(self, edge, source_node):
        self.incoming_edges.append((edge, source_node))

    def add_outgoing_edge(self, edge, target_node):
        self.outgoing_edges.append((edge, target_node))

    def update(self):
        self.receive()
        self.produce()
        self.distribute()

    def receive(self):
        for edge, _ in self.incoming_edges:
            self.inventory = min(self.max_inventory, self.inventory + edge.update(0))

    def produce(self):
        production = min(self.calculate_production(), self.max_inventory - self.inventory)
        self.inventory = min(self.max_inventory, self.inventory + production)
        self.last_production = production

    def distribute(self):
        if not self.outgoing_edges:
            return
        total_demand = sum(edge.quantity for edge, _ in self.outgoing_edges)
        if total_demand == 0:
            total_demand = len(self.outgoing_edges)  # Distribute evenly if no demand
        available = max(self.inventory, self.last_production)  # Always distribute something
        ratio = available / total_demand
        for edge, target_node in self.outgoing_edges:
            amount = max(1, int(edge.quantity * ratio))  # Always send at least 1
            self.inventory = max(0, self.inventory - amount)
            edge.update(amount)
            edge.calculate_cost(self)

    def calculate_production(self):
        raise NotImplementedError("Subclasses must implement calculate_production method")

class LeafNode(Node):
    def __init__(self, max_inventory, node_class, spline_points, cost_type):
        super().__init__(max_inventory, node_class, cost_type)
        x, y = zip(*spline_points)
        self.spline = interpolate.interp1d(x, y, kind='linear', fill_value='extrapolate')

    def calculate_production(self):
        return max(0, int(self.spline(time)))

class CombinerNode(Node):
    def __init__(self, max_inventory, node_class, a_coeffs, b_powers, cost_type):
        super().__init__(max_inventory, node_class, cost_type)
        self.a_coeffs = a_coeffs
        self.b_powers = b_powers

    def calculate_production(self):
        production = 0
        for (edge, _), a, b in zip(self.incoming_edges, self.a_coeffs, self.b_powers):
            production += a * (max(1, edge.quantity) ** b)  # Always use at least 1
        return max(1, int(production))  # Always produce at least 1

class SinkNode(Node):
    def __init__(self, consumption_rate, cost_type):
        super().__init__(math.inf, "sink", cost_type)
        self.consumption_rate = consumption_rate
        self.total_consumed = 0

    def update(self):
        self.receive()
        self.consume()

    def consume(self):
        consumed = min(self.inventory, self.consumption_rate)
        self.inventory -= consumed
        self.total_consumed += consumed
        self.last_production = consumed  # For consistency in reporting

    def calculate_production(self):
        return 0  # Sink nodes don't produce

class SupplyChain:
    def __init__(self):
        self.nodes = []
        self.edges = []

    def add_node(self, node):
        self.nodes.append(node)

    def add_edge(self, source, target, unit_price, initial_quantity, min_cost, max_cost):
        edge = Edge(unit_price, min_cost, max_cost)
        edge.initialize(initial_quantity)
        self.edges.append(edge)
        source.add_outgoing_edge(edge, target)
        target.add_incoming_edge(edge, source)

    def update(self):
        for node in self.nodes:
            node.update()

## Mock Supply Chain

In [3]:
def create_supply_chain():
    supply_chain = SupplyChain()

    # Create leaf nodes with random cost types
    leaf1 = LeafNode(200, "raw_material_1", [(0, 10), (5, 15), (10, 20)], random.choice(["fixed", "positive_dynamic", "negative_dynamic"]))
    leaf2 = LeafNode(250, "raw_material_2", [(0, 15), (5, 20), (10, 25)], random.choice(["fixed", "positive_dynamic", "negative_dynamic"]))
    leaf3 = LeafNode(220, "raw_material_3", [(0, 12), (5, 18), (10, 22)], random.choice(["fixed", "positive_dynamic", "negative_dynamic"]))
    supply_chain.add_node(leaf1)
    supply_chain.add_node(leaf2)
    supply_chain.add_node(leaf3)

    # Create intermediate nodes with random cost types
    intermediate_a1 = CombinerNode(300, "intermediate_a1", [0.8, 1.0], [1, 1.2], random.choice(["fixed", "positive_dynamic", "negative_dynamic"]))
    intermediate_a2 = CombinerNode(280, "intermediate_a2", [0.9, 1.1], [1.1, 1.3], random.choice(["fixed", "positive_dynamic", "negative_dynamic"]))
    supply_chain.add_node(intermediate_a1)
    supply_chain.add_node(intermediate_a2)

    intermediate_b1 = CombinerNode(300, "intermediate_b1", [0.8, 1.0], [1, 1.2], random.choice(["fixed", "positive_dynamic", "negative_dynamic"]))
    intermediate_b2 = CombinerNode(280, "intermediate_b2", [0.9, 1.1], [1.1, 1.3], random.choice(["fixed", "positive_dynamic", "negative_dynamic"]))
    supply_chain.add_node(intermediate_b1)
    supply_chain.add_node(intermediate_b2)

    # Create final combiner node with random cost type
    final_combiner = CombinerNode(350, "final_product", [0.7, 0.8, 0.9], [1, 1.1, 1.2], random.choice(["fixed", "positive_dynamic", "negative_dynamic"]))
    supply_chain.add_node(final_combiner)

    # Create sink node with random cost type
    sink = SinkNode(consumption_rate=250, cost_type=random.choice(["fixed", "positive_dynamic", "negative_dynamic"]))
    supply_chain.add_node(sink)

    # Add edges with cost ranges
    supply_chain.add_edge(leaf1, intermediate_a1, 10, 30, 5, 15)
    supply_chain.add_edge(leaf2, intermediate_a1, 15, 35, 8, 20)
    supply_chain.add_edge(leaf2, intermediate_a2, 12, 30, 6, 18)
    supply_chain.add_edge(leaf3, intermediate_a2, 14, 32, 7, 19)
    supply_chain.add_edge(intermediate_a1, intermediate_b1, 20, 40, 10, 25)
    supply_chain.add_edge(intermediate_a2, intermediate_b1, 22, 42, 11, 27)
    supply_chain.add_edge(intermediate_b1, final_combiner, 20, 40, 10, 25)
    supply_chain.add_edge(intermediate_b2, final_combiner, 22, 42, 11, 27)
    supply_chain.add_edge(leaf3, final_combiner, 18, 35, 9, 23)
    supply_chain.add_edge(final_combiner, sink, 25, 50, 13, 30)

    return supply_chain

In [4]:
# Simulation
time = 0
supply_chain = create_supply_chain()

for _ in range(200):  # Simulate for 200 time steps
    supply_chain.update()
    time += 1

    # Print current state
    print(f"Time: {time}")
    for node in supply_chain.nodes:
        print(f"Node ({node.node_class}): Inventory = {node.inventory}, Last Production = {node.last_production}, Cost Type = {node.cost_type}")
        for edge, target_node in node.outgoing_edges:
            print(f"  Edge to {target_node.node_class}: Quantity = {edge.quantity}, Cost = {edge.current_cost:.2f}")
    print("---")

Time: 1
Node (raw_material_1): Inventory = 0, Last Production = 10, Cost Type = positive_dynamic
  Edge to intermediate_a1: Quantity = 30, Cost = 5.00
Node (raw_material_2): Inventory = 1, Last Production = 15, Cost Type = fixed
  Edge to intermediate_a1: Quantity = 35, Cost = 14.00
  Edge to intermediate_a2: Quantity = 30, Cost = 12.00
Node (raw_material_3): Inventory = 1, Last Production = 12, Cost Type = positive_dynamic
  Edge to intermediate_a2: Quantity = 32, Cost = 7.38
  Edge to final_product: Quantity = 35, Cost = 9.06
Node (intermediate_a1): Inventory = 0, Last Production = 95, Cost Type = positive_dynamic
  Edge to intermediate_b1: Quantity = 40, Cost = 10.00
Node (intermediate_a2): Inventory = 0, Last Production = 137, Cost Type = positive_dynamic
  Edge to intermediate_b1: Quantity = 42, Cost = 11.00
Node (intermediate_b1): Inventory = 0, Last Production = 120, Cost Type = negative_dynamic
  Edge to final_product: Quantity = 40, Cost = 25.00
Node (intermediate_b2): Invento

## Recording of the simulation

In [5]:
import pandas as pd
import uuid

def simulate_and_collect_data(supply_chain, n_cycles):
    # Initialize DataFrames
    metadata = []
    node_data = []
    edge_data = []
    
    node_id = 0

    # Create metadata and assign unique IDs
    for node in supply_chain.nodes:
        metadata.append({
            'node_id': f"node_{node_id}",
            'node_class': node.node_class,
            'max_inventory': node.max_inventory,
            'cost_type': node.cost_type
        })
        node.id = node_id  # Assign ID to node object for reference
        node_id += 1

    edge_id = 0
    # Create edge metadata
    for edge in supply_chain.edges:
        source_node = None
        target_node = None
        
        for node in supply_chain.nodes:
            if edge in [e for e, _ in node.outgoing_edges]:
                source_node = node
            if edge in [e for e, _ in node.incoming_edges]:
                target_node = node
            if source_node and target_node:
                break
        
        if not source_node or not target_node:
            print(f"Warning: Edge {edge_id} is not properly connected.")
            print(f"Source node: {source_node.node_class if source_node else 'None'}")
            print(f"Target node: {target_node.node_class if target_node else 'None'}")
            continue
        
        metadata.append({
            'edge_id': f"edge_{edge_id}",
            'source_node_id': source_node.id,
            'target_node_id': target_node.id,
            'unit_price': edge.unit_price,
            'min_cost': edge.min_cost,
            'max_cost': edge.max_cost
        })
        edge.id = edge_id  # Assign ID to edge object for reference
        edge_id += 1

    # Simulate for n cycles
    for cycle in range(n_cycles):
        supply_chain.update()

        # Collect node data
        for node in supply_chain.nodes:
            node_data.append({
                'cycle': cycle,
                'node_id': f"node_{node.id}",
                'inventory': node.inventory,
                'last_production': node.last_production
            })

        # Collect edge data
        for edge in supply_chain.edges:
            if hasattr(edge, 'id'):  # Only collect data for edges that were properly connected
                edge_data.append({
                    'cycle': cycle,
                    'edge_id': f"edge_{edge.id}",
                    'quantity': edge.quantity,
                    'current_cost': edge.current_cost
                })

    # Create DataFrames
    metadata_df = pd.DataFrame(metadata)
    node_data_df = pd.DataFrame(node_data)
    edge_data_df = pd.DataFrame(edge_data)

    return metadata_df, node_data_df, edge_data_df

In [6]:
supply_chain = create_supply_chain()
metadata_df, node_data_df, edge_data_df = simulate_and_collect_data(supply_chain, n_cycles=200)

In [7]:
metadata_df

,node_id,node_class,max_inventory,cost_type,edge_id,source_node_id,target_node_id,unit_price,min_cost,max_cost
0,node_0,raw_material_1,200.0,negative_dynamic,NaN,NaN,NaN,NaN,NaN,NaN
1,node_1,raw_material_2,250.0,negative_dynamic,NaN,NaN,NaN,NaN,NaN,NaN
2,node_2,raw_material_3,220.0,fixed,NaN,NaN,NaN,NaN,NaN,NaN
3,node_3,intermediate_a1,300.0,negative_dynamic,NaN,NaN,NaN,NaN,NaN,NaN
4,node_4,intermediate_a2,280.0,negative_dynamic,NaN,NaN,NaN,NaN,NaN,NaN
5,node_5,intermediate_b1,300.0,fixed,NaN,NaN,NaN,NaN,NaN,NaN
6,node_6,intermediate_b2,280.0,fixed,NaN,NaN,NaN,NaN,NaN,NaN
7,node_7,final_product,350.0,negative_dynamic,NaN,NaN,NaN,NaN,NaN,NaN
8,node_8,sink,inf,fixed,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,edge_0,0.0,3.0,10.0,5.0,15.0


In [8]:
node_data_df

,cycle,node_id,inventory,last_production
0,0,node_0,0,200
1,0,node_1,1,215
2,0,node_2,1,174
3,0,node_3,0,95
4,0,node_4,0,137
...,...,...,...,...
1795,199,node_4,0,98
1796,199,node_5,0,0
1797,199,node_6,0,1
1798,199,node_7,0,0


In [9]:
edge_data_df

,cycle,edge_id,quantity,current_cost
0,0,edge_0,30,15.000
1,0,edge_1,35,15.200
2,0,edge_2,30,17.952
3,0,edge_3,32,13.000
4,0,edge_4,40,25.000
...,...,...,...,...
1995,199,edge_5,280,27.000
1996,199,edge_6,300,17.500
1997,199,edge_7,1,19.000
1998,199,edge_8,91,16.000


## Conversion between JSON and DataFrames

In [10]:
import pandas as pd
import json
from typing import Dict, List

def dataframes_to_json(metadata: pd.DataFrame, node: pd.DataFrame, edge: pd.DataFrame) -> str:
    bom = {
        "metadata": metadata.to_dict(orient='records'),
        "nodes": {},
        "edges": {}
    }

    # Process node data
    for _, row in node.iterrows():
        node_id = row['node_id']
        cycle = row['cycle']
        if node_id not in bom["nodes"]:
            bom["nodes"][node_id] = {}
        bom["nodes"][node_id][cycle] = row.to_dict()

    # Process edge data
    for _, row in edge.iterrows():
        edge_id = row['edge_id']
        cycle = row['cycle']
        if edge_id not in bom["edges"]:
            bom["edges"][edge_id] = {}
        bom["edges"][edge_id][cycle] = row.to_dict()

    return json.dumps(bom, indent=2)

def json_to_dataframes(json_data: str) -> Dict[str, pd.DataFrame]:
    bom = json.loads(json_data)

    # Create empty lists to store the data
    node_data = []
    edge_data = []

    # Process nodes
    for node_id, cycles in bom["nodes"].items():
        for cycle, node_info in cycles.items():
            node_data.append(node_info)

    # Process edges
    for edge_id, cycles in bom["edges"].items():
        for cycle, edge_info in cycles.items():
            edge_data.append(edge_info)

    # Create DataFrames
    metadata_df = pd.DataFrame(bom["metadata"])
    node_df = pd.DataFrame(node_data)
    edge_df = pd.DataFrame(edge_data)

    return {
        "metadata": metadata_df,
        "node": node_df,
        "edge": edge_df
    }

In [11]:
# Convert dataframes to JSON
json_string = dataframes_to_json(metadata_df, node_data_df, edge_data_df)

In [12]:
json_data = json.loads(json_string)
json_data

{'metadata': [{'node_id': 'node_0',
   'node_class': 'raw_material_1',
   'max_inventory': 200.0,
   'cost_type': 'negative_dynamic',
   'edge_id': nan,
   'source_node_id': nan,
   'target_node_id': nan,
   'unit_price': nan,
   'min_cost': nan,
   'max_cost': nan},
  {'node_id': 'node_1',
   'node_class': 'raw_material_2',
   'max_inventory': 250.0,
   'cost_type': 'negative_dynamic',
   'edge_id': nan,
   'source_node_id': nan,
   'target_node_id': nan,
   'unit_price': nan,
   'min_cost': nan,
   'max_cost': nan},
  {'node_id': 'node_2',
   'node_class': 'raw_material_3',
   'max_inventory': 220.0,
   'cost_type': 'fixed',
   'edge_id': nan,
   'source_node_id': nan,
   'target_node_id': nan,
   'unit_price': nan,
   'min_cost': nan,
   'max_cost': nan},
  {'node_id': 'node_3',
   'node_class': 'intermediate_a1',
   'max_inventory': 300.0,
   'cost_type': 'negative_dynamic',
   'edge_id': nan,
   'source_node_id': nan,
   'target_node_id': nan,
   'unit_price': nan,
   'min_cost': 

In [13]:
if SAVE_RECORDS:
    import json
    with open('../data/json/true.json', 'w') as f:
        json.dump(json_data, f, ensure_ascii=False)

In [14]:
# Convert JSON back to dataframes
reconstructed_dfs = json_to_dataframes(json_string)

# Access the reconstructed dataframes
reconstructed_metadata = reconstructed_dfs['metadata']
reconstructed_node = reconstructed_dfs['node']
reconstructed_edge = reconstructed_dfs['edge']

In [15]:
if SAVE_RECORDS:
    reconstructed_metadata.to_csv('../data/csv/metadata.csv', index=False)
    reconstructed_node.to_csv('../data/csv/node.csv', index=False)
    reconstructed_edge.to_csv('../data/csv/edge.csv', index=False)

In [16]:
reconstructed_metadata

,node_id,node_class,max_inventory,cost_type,edge_id,source_node_id,target_node_id,unit_price,min_cost,max_cost
0,node_0,raw_material_1,200.0,negative_dynamic,NaN,NaN,NaN,NaN,NaN,NaN
1,node_1,raw_material_2,250.0,negative_dynamic,NaN,NaN,NaN,NaN,NaN,NaN
2,node_2,raw_material_3,220.0,fixed,NaN,NaN,NaN,NaN,NaN,NaN
3,node_3,intermediate_a1,300.0,negative_dynamic,NaN,NaN,NaN,NaN,NaN,NaN
4,node_4,intermediate_a2,280.0,negative_dynamic,NaN,NaN,NaN,NaN,NaN,NaN
5,node_5,intermediate_b1,300.0,fixed,NaN,NaN,NaN,NaN,NaN,NaN
6,node_6,intermediate_b2,280.0,fixed,NaN,NaN,NaN,NaN,NaN,NaN
7,node_7,final_product,350.0,negative_dynamic,NaN,NaN,NaN,NaN,NaN,NaN
8,node_8,sink,inf,fixed,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,edge_0,0.0,3.0,10.0,5.0,15.0


In [17]:
reconstructed_node

,cycle,node_id,inventory,last_production
0,0,node_0,0,200
1,1,node_0,0,200
2,2,node_0,0,200
3,3,node_0,0,200
4,4,node_0,0,200
...,...,...,...,...
1795,195,node_8,19407,250
1796,196,node_8,19507,250
1797,197,node_8,19607,250
1798,198,node_8,19707,250


In [18]:
reconstructed_edge 

,cycle,edge_id,quantity,current_cost
0,0,edge_0,30,15.0
1,1,edge_0,200,15.0
2,2,edge_0,200,15.0
3,3,edge_0,200,15.0
4,4,edge_0,200,15.0
...,...,...,...,...
1995,195,edge_9,350,30.0
1996,196,edge_9,350,30.0
1997,197,edge_9,350,30.0
1998,198,edge_9,350,30.0


## Aggredation of Cycles and Conversion between JSON and Dataframes

In [19]:
import pandas as pd
import json
import random
from typing import Dict

def aggregated_dataframes_to_json(metadata: pd.DataFrame, node: pd.DataFrame, edge: pd.DataFrame, max_aggregation_cycles: int = 3) -> str:
    bom = {
        "metadata": metadata.to_dict(orient='records'),
        "nodes": {},
        "edges": {}
    }

    # Initialize variables to track the last cycle
    last_node_cycle = 0
    last_edge_cycle = 0
    
    num_cycles_to_aggregate = random.randint(1, max_aggregation_cycles)

    # Process node data with aggregation
    for _, row in node.iterrows():
        node_id = row['node_id']
        cycle = row['cycle']
        # Check if we can aggregate
        if node_id not in bom["nodes"]:
            bom["nodes"][node_id] = {}
        
        # Aggregate nodes
        for _ in range(num_cycles_to_aggregate):
            last_node_cycle += 1
            # Create a new entry for the aggregated node
            aggregated_node = row.copy()
            aggregated_node['cycle'] = last_node_cycle
            bom["nodes"][node_id][last_node_cycle] = aggregated_node.to_dict()

    # Process edge data with aggregation
    for _, row in edge.iterrows():
        edge_id = row['edge_id']
        cycle = row['cycle']
        # Check if we can aggregate
        if edge_id not in bom["edges"]:
            bom["edges"][edge_id] = {}
        
        # Aggregate edges
        for _ in range(num_cycles_to_aggregate):
            last_edge_cycle += 1
            # Create a new entry for the aggregated edge
            aggregated_edge = row.copy()
            aggregated_edge['cycle'] = last_edge_cycle
            bom["edges"][edge_id][last_edge_cycle] = aggregated_edge.to_dict()

    return json.dumps(bom, indent=2)

In [20]:
aggregated_json = aggregated_dataframes_to_json(metadata_df, node_data_df, edge_data_df)
aggregated_json_dict = json.loads(aggregated_json)

aggregated_json_dict

{'metadata': [{'node_id': 'node_0',
   'node_class': 'raw_material_1',
   'max_inventory': 200.0,
   'cost_type': 'negative_dynamic',
   'edge_id': nan,
   'source_node_id': nan,
   'target_node_id': nan,
   'unit_price': nan,
   'min_cost': nan,
   'max_cost': nan},
  {'node_id': 'node_1',
   'node_class': 'raw_material_2',
   'max_inventory': 250.0,
   'cost_type': 'negative_dynamic',
   'edge_id': nan,
   'source_node_id': nan,
   'target_node_id': nan,
   'unit_price': nan,
   'min_cost': nan,
   'max_cost': nan},
  {'node_id': 'node_2',
   'node_class': 'raw_material_3',
   'max_inventory': 220.0,
   'cost_type': 'fixed',
   'edge_id': nan,
   'source_node_id': nan,
   'target_node_id': nan,
   'unit_price': nan,
   'min_cost': nan,
   'max_cost': nan},
  {'node_id': 'node_3',
   'node_class': 'intermediate_a1',
   'max_inventory': 300.0,
   'cost_type': 'negative_dynamic',
   'edge_id': nan,
   'source_node_id': nan,
   'target_node_id': nan,
   'unit_price': nan,
   'min_cost': 

In [21]:
if SAVE_RECORDS:
    import json
    with open('../data/json/aggregated.json', 'w') as f:
        json.dump(aggregated_json_dict, f, ensure_ascii=False)

In [22]:
aggregated_dataframe = json_to_dataframes(aggregated_json)

aggregated_metadata = aggregated_dataframe["metadata"]
aggregated_nodes = aggregated_dataframe["node"]
aggregated_edges = aggregated_dataframe["edge"]

In [23]:
if SAVE_RECORDS:
    aggregated_metadata.to_csv("../data/csv/aggregated_metadata.csv", index=False)
    aggregated_nodes.to_csv("../data/csv/aggregated_node_data.csv", index=False)
    aggregated_edges.to_csv("../data/csv/aggregated_edge_data.csv", index=False)

In [24]:
aggregated_metadata

,node_id,node_class,max_inventory,cost_type,edge_id,source_node_id,target_node_id,unit_price,min_cost,max_cost
0,node_0,raw_material_1,200.0,negative_dynamic,NaN,NaN,NaN,NaN,NaN,NaN
1,node_1,raw_material_2,250.0,negative_dynamic,NaN,NaN,NaN,NaN,NaN,NaN
2,node_2,raw_material_3,220.0,fixed,NaN,NaN,NaN,NaN,NaN,NaN
3,node_3,intermediate_a1,300.0,negative_dynamic,NaN,NaN,NaN,NaN,NaN,NaN
4,node_4,intermediate_a2,280.0,negative_dynamic,NaN,NaN,NaN,NaN,NaN,NaN
5,node_5,intermediate_b1,300.0,fixed,NaN,NaN,NaN,NaN,NaN,NaN
6,node_6,intermediate_b2,280.0,fixed,NaN,NaN,NaN,NaN,NaN,NaN
7,node_7,final_product,350.0,negative_dynamic,NaN,NaN,NaN,NaN,NaN,NaN
8,node_8,sink,inf,fixed,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,edge_0,0.0,3.0,10.0,5.0,15.0


In [25]:
aggregated_nodes

,cycle,node_id,inventory,last_production
0,1,node_0,0,200
1,2,node_0,0,200
2,3,node_0,0,200
3,28,node_0,0,200
4,29,node_0,0,200
...,...,...,...,...
5395,5372,node_8,19707,250
5396,5373,node_8,19707,250
5397,5398,node_8,19807,250
5398,5399,node_8,19807,250


In [26]:
aggregated_edges

,cycle,edge_id,quantity,current_cost
0,1,edge_0,30,15.0
1,2,edge_0,30,15.0
2,3,edge_0,30,15.0
3,31,edge_0,200,15.0
4,32,edge_0,200,15.0
...,...,...,...,...
5995,5969,edge_9,350,30.0
5996,5970,edge_9,350,30.0
5997,5998,edge_9,350,30.0
5998,5999,edge_9,350,30.0
